# Modellering XGBoost

in dit verslag neem ik je mee in hoe ik het XGBoost model getrained heb en welke bijzonderheden ik tegen kwam

In [33]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path
current_dir = Path.cwd()
project_dir = current_dir.parent.parent
sys.path.append(str(project_dir))
from pipeline import prepare_data, undersample, markeer_omslagpunten
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


eerst moeten we onze data inlezen.

In [34]:
df = pd.read_csv("../../data/export_location_modeling/export_location_modeling.csv")

dan moeten we de data voorbereiden voor het modelleren, dit word gedaan met behulp van de pipeline en haalt voornamelijk meet fouten en rare waardes er uit

In [35]:
df = prepare_data(df)
df = markeer_omslagpunten(df)

Totaal gedetecteerde files: 698
-----------------------------------------------------
Detected files (unique file_id): 698
Marked omslagpunten:             698
-----------------------------------------------------


eerst moeten we de data splitsen in een train en test set zodat we kunnen checken of het model het goed doet

In [36]:
X = df[["tijd",'gem_intensiteit_smooth' , "day_of_week"]]
y = df['file_omslag_flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

laten we dan even snel een simpel model maken en kijken hoe die het doet

In [37]:
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


hier is te zien dat het model het eigenlijk heel slecht doet dit komt waarschijnlijk omdat er veel minder files zijn dan niet files

In [38]:
y_pred = DTC.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Recall: %.2f%%" % (recall * 100.0))
print("Precision: %.2f%%" % (precision * 100.0))

Accuracy: 99.42%
Recall: 9.92%
Precision: 7.79%


laten we kijken of deze aanname correct is door naar de verdeling te kijken. hier is inderdaad te zien dat de verdeling erg extreem is, laten we kijken of we dit op kunnen lossen

In [39]:
target_count = df.file_omslag_flag.value_counts()
print('geen file :', target_count[0])
print('wel file:', target_count[1])
print('Proportion:', round(target_count[0] / target_count[1], 2), ': 1')

target_count.plot(kind='bar', title='Count (target)', xlabel=("geen file","                                  " "wel file"));

geen file : 213928
wel file: 698
Proportion: 306.49 : 1


onderstaande functie zorgt er voor dat de data geundersampled word, dit betekend dat we wel alle data houden waar er wel files zijn maar de data waar geen files zijn verminderen zodat de ratio tussen de twee beter is en we mogelijk een beter model kunnen trainen

In [40]:

def undersample(df, minority_class=1, ratio=3, random=17):
    df_minority = df[df["file_omslag_flag"] == minority_class].copy()
    df_majority = df[df["file_omslag_flag"] != minority_class].copy()
    n_minority = len(df_minority)
    n_majority_target = int(n_minority * ratio)
    df_majority_undersampled = df_majority.sample(n=n_majority_target,random_state=random)
    df_undersampled = pd.concat([df_minority, df_majority_undersampled], ignore_index=True)    
    return df_undersampled




hieronder splitsen we de de data in 3 delen, de train en test set die we gebruiken om het model te trainen en daarnaast een validatie set, deze is niet geundersampled zodat we kunnen kijken of het model het na het trainen ook goed doet op "normale" data

In [41]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score



X = df[["tijd",'gem_intensiteit_smooth' , "day_of_week"]]
y = df['file_omslag_flag']

val_train, b, val_test, c = train_test_split(X, y, test_size=0.2, random_state=1)

print(y.value_counts())
df = undersample(df, ratio=3)

X = df[["tijd",'gem_intensiteit_smooth' , "day_of_week"]]
y = df['file_omslag_flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(y_train.value_counts())

file_omslag_flag
0    213928
1       698
Name: count, dtype: int64
file_omslag_flag
0    1674
1     559
Name: count, dtype: int64


hier starten we eerst weer met het maken van een model. in dit geval kies ik voor een XGBoost model dit is een descision tree die zich zelf verbeterd

In [42]:
model = XGBClassifier()
model.fit(X_train, y_train)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


de resultaten van het model op de kleinere dataset zijn redelijk dit kan mogelijk nog verbeteren door de hyperparamenters mee te geven en deze te tunen maar laten we eerst eens kijken hoe het model het doet op de validatie set.

In [43]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Recall: %.2f%%" % (recall * 100.0))
print("Precision: %.2f%%" % (precision * 100.0))

Accuracy: 84.97%
Recall: 68.35%
Precision: 70.37%


hieronder is te zien dat het model het erg slecht doet op de validatie set er word namelijk is 97% van de gevallen dat er file voorspeld is een fout gemaakt. hier is niet mee te werken en we kunnen dus redelijk goed concluderen dat met alleen deze data er geen voorspelling gedaan kan worden die nuttig is

In [44]:
y_pred = model.predict(val_train)

accuracy = accuracy_score(val_test, y_pred)
recall = recall_score(val_test, y_pred)
precision = precision_score(val_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Recall: %.2f%%" % (recall * 100.0))
print("Precision: %.2f%%" % (precision * 100.0))

Accuracy: 91.35%
Recall: 89.08%
Precision: 3.36%


In [45]:
y_pred = model.predict(val_train)
print(classification_report(val_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95    171123
           1       0.03      0.89      0.06       577

    accuracy                           0.91    171700
   macro avg       0.52      0.90      0.51    171700
weighted avg       1.00      0.91      0.95    171700



zie hieronder de confusion matrix op de validatie set, hierin is te zien dat het model veel te vaak file voorspeld als dit niet zo is

In [46]:
ConfusionMatrixDisplay.from_predictions(
    val_test,
    y_pred,
    display_labels=["geen file", "wel file"],
    cmap="Blues",
    values_format="d"
)